# Håndter ordliste

Denne notatboken inneholder en oppskrift på tolkning av rå copy-paste fra
Sharepoint og overføring til BigQuery for strukturert aksess.

---

NAV Kontaktsenter holder en [ordliste med oversettelser fra norsk til engelsk på
Sharepoint](https://navno.sharepoint.com/sites/enhet-kontaktsenter/SitePages/Ordliste-Norsk-Engelsk.aspx).

Ordlisten kan kopieres inn med copy-past, men kan kreve litt manuelt arbeid for
å kunne automatisk prosessere. Notatboken vil varsle hvilke linjer som må fikses
hvis den ikke kan tolke ordlisten.

Notatboken forventer følgende format (**merk** at det er `<tab>` mellom ordet og
oversettelsen):

```raw
<norsk>\t<engelsk>\n
```

In [ ]:
# Parametere for notatboken
verbose = False
"""Skriv ut ekstra informasjon"""

# BigQuery parametere
project = "nks-aiautomatisering-prod-194a"
dataset = "kunnskapsbase"
table = "ordliste"

In [ ]:
# Oppsett for Rich slik at vi kan printe ut med farger
from rich.console import Console

console = Console()

## Les inn rådata

In [ ]:
import pathlib

data_folder = pathlib.Path("../data")
potential = list(data_folder.glob("ordliste_*.txt"))
# TODO: La bruker velge selv, akkurat nå tar vi den nyeste
selected = sorted(potential)[0]

Vi antar at hver linje inneholder en oversettelse med det norske ordet skilt fra
det engelske med en `<tab>`.

In [ ]:
raw_list = open(selected).read()
raw_list = raw_list.split("\n")
raw_list = [trans.split("\t", maxsplit=1) for trans in raw_list if trans.strip()]

Etter prosessering, hvis en linje ikke inneholder 2 termer så ber vi bruker om å
fikse disse linjene.

In [ ]:
need_fix = [
    (i, trans)
    for i, trans in enumerate(raw_list)
    if len(trans) == 1 or any([e.strip() == "" for e in trans])
]
if need_fix:
    from rich.table import Table

    console.print(f"Følgende oversettelser må fikses i [magenta]'{selected}'[/]:")
    fix_table = Table("Linje", "Tekst på linjen")
    for idx, word in need_fix:
        fix_table.add_row(str(idx), word[0])
    console.print(fix_table)
assert not need_fix, "Ordliste må fikses før den kan prosesseres"

## Liste med oversettelser

In [ ]:
translation = [(nor.strip(), eng.strip()) for nor, eng in raw_list]
console.print(
    f"Fant [blue bold]{len(translation)}[/] oversettelser i ordliste [magenta]{selected}[/]"
)

In [ ]:
from rich.table import Table

trans_table = Table("Norsk", "Engelsk", title="NKS Ordliste")
for nor, eng in translation:
    trans_table.add_row(nor, eng)
if verbose:
    console.print(trans_table)

## Opplasting til BigQuery

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=project)
table_id = f"{project}.{dataset}.{table}"

Før opplasting til BigQuery må vi passe på at skjemaet på BigQuery er definert
og slik vi forventer.

In [ ]:
from google.cloud.bigquery import SchemaField

# Database skjema på BigQuery
schema = [
    SchemaField(
        "uploaded", "DATETIME", mode="REQUIRED", description="Tidspunkt for opplasting"
    ),
    SchemaField(
        "norwegian",
        field_type="STRING",
        mode="REQUIRED",
        description="Det norske ordet",
    ),
    SchemaField(
        "english",
        field_type="STRING",
        mode="REQUIRED",
        description="Den engelske oversettelsen",
    ),
]

In [ ]:
# Opprett database hvis den ikke allerede finnes
table_ref = bigquery.TableReference.from_string(table_id=table_id)
table = client.create_table(table_ref, exists_ok=True)
if not table.schema:
    table.schema = schema
    client.update_table(table, fields=["schema"])
    console.print(f"Opprettet/Endret database '{table.full_table_id}'")
else:
    console.print("Spørsmålsdatabase eksisterer allerede")

Når vi vet at tabellen eksisterer så kan vi forberede data for opplasting.

In [ ]:
import datetime

upload = datetime.datetime.now(datetime.UTC)
data_to_upload = [
    {"uploaded": upload, "norwegian": nor, "english": eng} for nor, eng in translation
]

In [ ]:
from rich.prompt import Confirm

if Confirm.ask(
    f"Vil du laste opp [bold blue]{len(data_to_upload)}[/] rader?", console=console
):
    errors = client.insert_rows(
        table=table_id, rows=data_to_upload, selected_fields=schema
    )
    if errors:
        console.print("[bold red]Opplasting av ordliste feilet!")
        console.print(f"\tFeil: {errors}")
    else:
        console.print("[bold green]Opplasting fullført!")
else:
    console.print("[yellow bold]Hoppet over opplasting")